# Calculate entrainment adjustment

Anna Mackie, 2024

This script calculates the entraintment adjustment (see section 3.2 and Text S1) for the control simulation, using the zero-buoyancy plume model from Singh & O'Gorman (2013).

Integrates the ZBP model between 500hPa and the LCL. In p coordinates:

$\frac{dh^*}{dp} =  -\frac{\hat{\epsilon }}{p\ln(p/p_0)} \frac{T_v(p)}{\{T_v(p)\}} L_v[q^*(p) - q(p)]$

where $\{T_v(p)\}\equiv \int_{p_0}^p(T_v/p)dp / \int_{p_0}^p(1/p)dp$ is a virtual temperature inversely weighted by pressure between the surface and a given pressure level.

It then finds the optimal entrainment parameter ($\hat{\epsilon}$) to fit the ascent fraction as calculated by the instability index to that as calculated by the vertical velocity at 500hPa.

For a particular gridpoint $i,j$, the instability index is :

$[\Phi^e]_{i,j} = [h_{sfc}]_{i,j} - \hat{\epsilon}[h^{*e}]_{i,j} - [h^{*}_{500}]_{i,j}$

the routine finds an $\hat{\epsilon}$ such that the area-weighted proportion of points with a poistive instability index over the 12 months of control simulation (at a monthly frequency) matches the area-weighted proportion of gridpoints with negative vertical velocity (ascending) at 500 hPa.

Finally, $[h^{*e}]_{i,j}$ is calculated for the different perturbation simulations. All 'entrainment adjustments' are saved for use in plotting scripts.

In [56]:
import sys
import pathlib
import platform
venv_path = '~/nb-venvs/metpy_venv'
sys.path.append(str(pathlib.Path(f'{venv_path}/lib/python{platform.python_version_tuple()[0]}.{platform.python_version_tuple()[1]}/site-packages/').expanduser()))

import metpy
print(metpy.__file__)

/opt/jaspy/lib/python3.11/site-packages/metpy/__init__.py


In [57]:
import numpy as np
import numpy.ma as ma
import cf
import matplotlib.pyplot as plt
from matplotlib import colors
import xarray as xr
from mpl_toolkits.basemap import Basemap
import matplotlib as mpl
import cartopy.crs as ccrs
from copy import copy
import glob
from scipy import stats
from scipy.optimize import minimize
from scipy.optimize import dual_annealing
from scipy.ndimage import gaussian_filter
import metpy.calc as mc
from metpy.units import units
import sys
#sys.path.append('../')

import regionmask
import matplotlib.gridspec as gridspec


plt.rcParams.update({'font.size': 14})

lets = ['a)', 'b)', 'c)', 'd)', 'e)', 'f)', 'g)', 'h)', 'i)', 'j)', 'k)', 'l)']    

sst_inc = ['-4', '-3', '-2', '-1','+1', '+2', '+3', '+4']
sst_dig = [-4, -3, -2, -1, 1, 2, 3, 4]
patches = ['100E', '140E', '180E', '220E']
cols = ['steelblue', 'deepskyblue', 'lightskyblue', 'powderblue','lightsalmon', 'darksalmon', 'tomato', 'red']#'
patch_cols = ['red', 'green', 'orange','blue']
markers = ['o', 'd']
lines = ['-', '--']

def proportion_overcome_conv_threshold(hsfc, hsat500, weights, landmask):
    """ proportion of domain ascending using instability
    index, including area weighting
    """
    lon, month = hsfc.lon, hsfc.month
    weights = weights.where(landmask.notnull()==False)
    if np.ndim(hsfc) ==3:
        weights_3Darray = (weights * xr.DataArray(np.ones(len(month)), dims=("month"))).T * xr.DataArray(np.ones(len(lon)), dims=("lon"))
    else:
        weights_3Darray = (weights * xr.DataArray(np.ones(len(lon)), dims=("lon")))
    instab_index = hsfc - hsat500
    return weights_3Darray.where(instab_index > 0).sum() / weights_3Darray.sum()

def calc_ascent_frac(wap500, weights, landmask):
    """ proportion of domain ascending using vertical velocity
    at 500 hPa, including area weighting
    """
    lon, month = wap500.lon, wap500.month
    weights = weights.where(landmask.notnull()==False)
    if np.ndim(wap500) ==3:
        weights_3Darray = (weights * xr.DataArray(np.ones(len(month)), dims=("month"))).T * xr.DataArray(np.ones(len(lon)), dims=("lon"))
    else:
        weights_3Darray = (weights * xr.DataArray(np.ones(len(lon)), dims=("lon")))
                           
    return weights_3Darray.where(wap500 < 0).sum() / weights_3Darray.sum()

def calc_prop(arr, weights, landmask):
    lon, month = arr.lon, arr.month
    weights = weights.where(landmask.notnull()==False)
    if np.ndim(arr) ==3:
        weights_3Darray = (weights * xr.DataArray(np.ones(len(month)), dims=("month"))).T * xr.DataArray(np.ones(len(lon)), dims=("lon"))
    else:
        weights_3Darray = (weights * xr.DataArray(np.ones(len(lon)), dims=("lon")))
    return float(weights_3Darray.where(arr.notnull()==True).sum() / weights_3Darray.sum())




## Constants

In [58]:
# constants
l = 20
g = metpy.constants.earth_gravity
cp = metpy.constants.dry_air_spec_heat_press
Lv = metpy.constants.water_heat_vaporization
Rv = metpy.constants.water_gas_constant
Md = metpy.constants.dry_air_molecular_weight
R0 = metpy.constants.dry_air_gas_constant
sst_change = [-4,-3,-2,-1,1,2,3,4]
no_patches = len(patches)
no_temps = len(sst_dig)


## Control simulation

In [59]:
# make landmask and latitude weighting from control
ds = xr.open_dataset('data/control_2d.nc')
ds3d = xr.open_dataset('data/control_3d.nc')

lat = ds.lat.sel(lat = slice(-l,l))

#define land mask
landmask = regionmask.defined_regions.natural_earth_v5_0_0.land_110.mask(ds.lon, lat)
weights = np.cos(np.deg2rad(lat))

SST = ds.ts.sel(lat = slice(-l, l)).where(landmask.notnull()==False)

In [60]:
# load control data
wap_ctrl = ds3d.wap.sel(lat = slice(-l,l))
wap500_ctrl =wap_ctrl.sel(plev = 50000.) * units.Pa /units.second

#MSE
zg = ds3d.zg.sel(lat = slice(-l, l)) *units.m
ta = ds3d.ta.sel(lat = slice(-l, l)) *units.K
q = ds3d.hus.sel(lat = slice(-l, l))*units.kilogram / units.kilogram
rh = ds3d.hur.sel(lat = slice(-l, l))
pfull = ds3d.pfull.sel(lat = slice(-l, l)) *units.Pa
ps = ds.ps.sel(lat = slice(-l,l)) * units.Pa
ts_ctrl = ds.ts.sel(lat = slice(-l,l)) * units.K
tas = ds.tas.sel(lat = slice(-l,l)) * units.K
dew2 = ds.dew2.sel(lat = slice(-l,l))* units.K
q2m = mc.specific_humidity_from_dewpoint(ps, dew2)

# calc MSE
h_ctrl = mc.moist_static_energy(zg, ta, q)
hsfc_ctrl = mc.moist_static_energy(2*units.m, tas, q2m)

# calc MSE*
sat_mixing_ratio = mc.saturation_mixing_ratio(pfull, ta)
sat_spec_hum = mc.specific_humidity_from_mixing_ratio(sat_mixing_ratio)
hsat_ctrl = mc.moist_static_energy(zg, ta, sat_spec_hum)
hsat500_ctrl = hsat_ctrl.sel(plev = 50000.)#.sel(plev = slice(30000., 60000.)).weighted(weights).mean(dim = ('plev'))

month, lat, lon = ta.month, ta.lat, ta.lon
plev_array = np.asarray(ta.plev)

#will integrate between 500hPa and LCL
#find LCL
lcl = mc.lcl(ps, tas, dew2) # returns a tuple: first dimension is pressure, the second temp
lcl_p = lcl[0]

sat_def = sat_spec_hum - q

plev_FT = 50000. # take this as free-tropospheric ref point
plev_FT_index = np.argmin(np.abs(plev_array - plev_FT))#index of 500hPa level


## Set up ZBP model equation and integrate

In [40]:
# calculate the virtual tempertaure factor {Tv(p)} 
plev_array= np.asarray(h_ctrl.plev)
mixing_ratio = mc.mixing_ratio_from_specific_humidity(q)
virtual_temp = mc.virtual_temperature(ta, mixing_ratio)
virtual_temp_by_p = virtual_temp/pfull
part1 = np.empty(np.shape(virtual_temp_by_p))
part2 = np.empty(np.shape(virtual_temp_by_p))

#restrict range of calculation to between 500hPa and 1000hPa
for plev_index in range(len(plev_array)):
    pressure = plev_array[plev_index]
    # restrict from pressure to p0 
    part1i = virtual_temp_by_p.sel(plev = slice(pressure, 100000.))               
    part2i = (1/pfull).sel(plev = slice(pressure, 100000.)) 
    
    if plev_array[plev_index] == plev_array[-1]: # first plevel, if only one level
        part1[:,plev_index,] = part1i[:,0,] 
        part2[:,plev_index,] = part2i[:,0,]
    else: # integrate over other levels, stack
        part1[:,plev_index,] = part1i.integrate(coord = 'plev')
        part2[:,plev_index,] = part2i.integrate(coord = 'plev')

virtual_temperature_pressure_weighted = part1/part2

In [48]:
# put this inequation to integrate
dhdp = (virtual_temp/virtual_temperature_pressure_weighted)*(-Lv*sat_def)/(pfull*np.log(pfull/ps))

#entrainment calc - no need to re-run unless routine changes as have saved output.
# this uses both land and ocean points

adjustment = np.empty((12,len(lat), len(lon))) # empty array to fill
hlcl = np.empty((12,len(lat), len(lon)))
for month in range(12):
    for lati in range(len(lat)):
        for loni in range(len(lon)):
            lcli = np.asarray(lcl_p[month, lati, loni]) # get lcl of that lat/lon/month
            lcl_plev = np.argmin(np.abs(plev_array - lcli)) # find index of closest plev to lcli
            dhdp_500_to_lcl = dhdp.sel(month = month+1, lat = lat[lati], lon = lon[loni], plev = slice(plev_FT, plev_array[lcl_plev])) # restrict dhdz to plevels above lcl and below 500 hPa
            plev_500_to_lcl = plev_array[plev_FT_index:lcl_plev+1] # get array of pressure levels integrating over
            adjustment[month,lati, loni] = np.trapz(dhdp_500_to_lcl, x = plev_500_to_lcl) # numerically integrate that lat/lon/month column
            hlcl[month, lati, loni] = h_ctrl.sel(month = month+1, lat = lat[lati], lon = lon[loni], plev = h_ctrl.plev[lcl_plev])
np.save('entrainment_adjustments/adjustment_ctrl.npy', adjustment, allow_pickle = True)

## Optimisation to obtain epsilon

In [61]:
#load control adjustment due to entrainment
months = np.arange(1,13,1)
adjustment_ctrl = np.load('entrainment_adjustments/adjustment_ctrl.npy')
adjustment_ctrl = xr.DataArray(adjustment_ctrl, dims=("month", "lat", "lon"), coords = (months, lat, lon))
adjustment_ctrl = adjustment_ctrl *units.joule / units.kilogram


In [63]:
# function of what I want minimised: in this case, the difference between the proportion overcoming 
# the convective threshold over the 12 months of the control simulation, and the ascent fraction,  
# in order to tune the entrainment parameter 

#need to adjust landmask, as some coastal gridpoints have adjustment_ctrl = NaN as they don't have values for 1000hPa
landmask = regionmask.defined_regions.natural_earth_v5_0_0.land_110.mask(ds.lon, lat)
landmask = xr.ones_like(wap500_ctrl).where(landmask.notnull()==False).where(adjustment_ctrl.notnull()==True)
landmask = xr.zeros_like(landmask).where(landmask !=1)

# this function returns the difference between the ascent fraction as determined by the instability index and vertical velocity
def func(ehat, hsfc, adjustment, hsat500, wap500): #arguments needs to be a tuple of the fixed params
    index = proportion_overcome_conv_threshold(hsfc - adjustment*ehat, hsat500, weights, landmask)
    alpha_up = calc_ascent_frac(wap500, weights, landmask)
    return np.abs(alpha_up - index)
    
# this routine minimisies the result from 'func' ie the difference (on landmasked points)
res = dual_annealing(func, bounds = [(0.1, 0.2)], 
                     args = (hsfc_ctrl.where(landmask.notnull()==False), adjustment_ctrl.where(landmask.notnull()==False), hsat500_ctrl.where(landmask.notnull()==False), 
                             wap500_ctrl.where(landmask.notnull()==False)),
                     maxiter = 1000) # default max iteration is 1000
ehat = res.x
print('entrainment parameter optimised to be: ' ,ehat)


entrainment parameter optimised to be:  [0.17746965]


## Perturbation experiments

Repeat the process done with the control simulation (ZBP model) for the perturbation simulations and save

In [ ]:
alpha_up_array, orig_array, adjust_array = np.empty((4,8)),np.empty((4,8)),np.empty((4,8))

plev_FT = 50000.

for k in range(4):
    
    patch = patches[k]
    print('patch = ', patch)
    for j in range(len(sst_inc)):
        delta_sst = sst_inc[j]
        print('delta sst =  ', delta_sst)
        ds1_3d = xr.open_dataset('data/' + patch +'_0N_' + delta_sst + 'K_3d.nc')
        ds1 = xr.open_dataset('data/' + patch +'_0N_' + delta_sst + 'K_2d.nc')
    
        
        wap = ds1_3d.wap.sel(lat = slice(-l,l))
        wap500 =wap.sel(plev = 50000.) * units.Pa /units.second
        lat = wap500.lat
        lon = wap500.lon
        plev_array = np.asarray(wap.plev)

        plev_FT_index = np.argmin(np.abs(plev_array - plev_FT))
        
        #MSE
       
        zg = ds1_3d.zg.sel(lat = slice(-l, l)) *units.m
        ta = ds1_3d.ta.sel(lat = slice(-l, l)) *units.K
        q = ds1_3d.hus.sel(lat = slice(-l, l))*units.kilogram / units.kilogram
        rh = ds1_3d.hur.sel(lat = slice(-l, l))
        pfull = ds1_3d.pfull.sel(lat = slice(-l, l)) *units.Pa
        ps = ds1.ps.sel(lat = slice(-l,l)) * units.Pa
        ts = ds1.ts.sel(lat = slice(-l,l)) * units.K
        tas = ds1.tas.sel(lat = slice(-l,l)) * units.K
        dew2 = ds1.dew2.sel(lat = slice(-l,l))* units.K
        q2m = mc.specific_humidity_from_dewpoint(ps, dew2)
        
        #MSE
        h = mc.moist_static_energy(zg, ta, q)
        h500 = h.sel(plev=50000.)
        hsfc = mc.moist_static_energy(2*units.m, tas, q2m)
        
        #MSE*
        sat_mixing_ratio = mc.saturation_mixing_ratio(pfull, ta)
        sat_spec_hum = mc.specific_humidity_from_mixing_ratio(sat_mixing_ratio)
        hsat = mc.moist_static_energy(zg, ta, sat_spec_hum)
        hsat500 = hsat.sel(plev = 50000.)
        
        #entrainment calc
        
        #lcl
        dewpoint = mc.dewpoint_from_specific_humidity(pfull, ta, q)
        
        lcl = mc.lcl(ps, tas, dew2) 
        lcl_p = lcl[0]
        
        sat_def = sat_spec_hum - q

        mixing_ratio = mc.mixing_ratio_from_specific_humidity(q)
        virtual_temp = mc.virtual_temperature(ta, mixing_ratio)
        virtual_temp_by_p = virtual_temp/pfull
        part1 = np.empty(np.shape(virtual_temp_by_p))
        part2 = np.empty(np.shape(virtual_temp_by_p))
        
        
        for plev_index in range(len(plev_array)):
            pressure = plev_array[plev_index]
            part1i = virtual_temp_by_p.sel(plev = slice(pressure, 100000.))  
            part2i = (1/pfull).sel(plev = slice(pressure, 100000.)) 

            if plev_array[plev_index] == plev_array[-1]:
                part1[:,plev_index,] = part1i[:,0,]
                part2[:,plev_index,] = part2i[:,0,]
            else:
                part1[:,plev_index,] = part1i.integrate(coord = 'plev')
                part2[:,plev_index,] = part2i.integrate(coord = 'plev')
                 
        virtual_temperature_pressure_weighted = part1/part2

        dhdp = (virtual_temp/virtual_temperature_pressure_weighted)*(-Lv*sat_def)/(pfull*np.log(pfull/ps))
        
        #numerically integrate
        #loop over lat/lon/month
        months = ta.month
        adjustment = np.empty((12,len(lat), len(lon))) # empty array to fill
        hlcl = np.empty((12,len(lat), len(lon)))
        for month in range(12):
            for lati in range(len(lat)):
                for loni in range(len(lon)):
                    lcli = np.asarray(lcl_p[month,lati, loni]) # get lcl of that lat/lon/month
                    lcl_plev = np.argmin(np.abs(plev_array - lcli)) # find index of closest plev to lcli
                    dhdp_500_to_lcl = dhdp.sel(month = month+1, lat = lat[lati], lon = lon[loni], plev = slice(plev_FT, plev_array[lcl_plev])) # restrict dhdz to plevels above lcl and below 500 hPa
                    plev_500_to_lcl = plev_array[plev_FT_index:lcl_plev+1] # get array of pressure levels integrating over
                    adjustment[month,lati, loni] = np.trapz(dhdp_500_to_lcl, x = plev_500_to_lcl) # numerically integrate that lat/lon/month column

        #choose ascent regions, calculate 
        np.save('entrainment_adjustments/adjustment'+ patch +'_0N_' + delta_sst + 'K.npy', adjustment, allow_pickle=True)


patch =  100E
delta sst =   -4


/tmp/ipykernel_15451/2589029043.py:72: RuntimeWarning: invalid value encountered in divide
  virtual_temperature_pressure_weighted = part1/part2


delta sst =   -3


/tmp/ipykernel_15451/2589029043.py:72: RuntimeWarning: invalid value encountered in divide
  virtual_temperature_pressure_weighted = part1/part2


delta sst =   -2


...

/tmp/ipykernel_15451/2589029043.py:72: RuntimeWarning: invalid value encountered in divide
  virtual_temperature_pressure_weighted = part1/part2


patch =  180E
delta sst =   -4


/tmp/ipykernel_15451/2589029043.py:72: RuntimeWarning: invalid value encountered in divide
  virtual_temperature_pressure_weighted = part1/part2


delta sst =   -3


/tmp/ipykernel_15451/2589029043.py:72: RuntimeWarning: invalid value encountered in divide
  virtual_temperature_pressure_weighted = part1/part2
